In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve

train = pd.read_csv('./data/input/train.csv', parse_dates=['datetime'], index_col='datetime')
test = pd.read_csv('./data/input/test.csv', parse_dates=['datetime'], index_col='datetime')
data = pd.concat([train, test], axis=0).sort_index()

# データ前処理

In [2]:
data_0 = data.copy()
data_0['year'] = data_0.index.year
data_0['month'] = data_0.index.month
# data['day'] = data.index.day
data_0['hour'] = data_0.index.hour
data_0['weekday'] = data_0.index.weekday

train_0 = data_0.loc[train.index, :]
test_0 = data_0.loc[test.index, :].drop(columns=['count', 'casual', 'registered'])

# 仮説検証

In [25]:
rf = RFR()
rf.fit(train_0.drop(columns=['count', 'casual', 'registered']), train_0['count'])
pred = rf.predict(test_0)

In [ ]:
submission = pd.DataFrame({'datetime': test_0.index, 'count': pred})
submission.to_csv('./data/output/t_0.csv', index=False)

## 仮説 1
count を registered と casual の和として予測する．

### t_1_1 : count

In [3]:
rf = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf.fit(train_0.drop(columns=['casual', 'registered', 'count']), train_0.loc[:, 'count'])
pred = rf.predict(test_0)

In [4]:
submission = pd.DataFrame({'datetime': test_0.index, 'count': pred})
submission.to_csv('./data/output/t_1_1.csv', index=False)

### t_1_2 : registered + casual

In [5]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_0.drop(columns=['casual', 'registered', 'count']), train_0.loc[:, 'registered'])
pred_r = rf_r.predict(test_0)

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_0.drop(columns=['casual', 'registered', 'count']), train_0.loc[:, 'casual'])
pred_c = rf_c.predict(test_0)

pred = pred_r + pred_c

In [6]:
submission = pd.DataFrame({'datetime': test_0.index, 'count': pred})
submission.to_csv('./data/output/t_1_2.csv', index=False)

### R: 1_2

## 仮説 2 : windspeed
風速 0 について

### 仮説 2-1 : delete
風速 を削除

In [7]:
# delete windspeed
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_0.drop(columns=['casual', 'registered', 'count', 'windspeed']), train_0.loc[:, 'registered'])
pred_r = rf_r.predict(test_0.drop(columns=['windspeed']))

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_0.drop(columns=['casual', 'registered', 'count', 'windspeed']), train_0.loc[:, 'casual'])
pred_c = rf_c.predict(test_0.drop(columns=['windspeed']))

pred = pred_r + pred_c

In [10]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_2_1.csv', index=False)

### 仮説 2-2 : mean
風速 0 を平均値に置き換え

In [9]:
data_2_2 = data_0.copy()
data_2_2.loc[data_2_2.loc[:, 'windspeed'] == 0, 'windspeed'] = data_2_2.loc[data_2_2.loc[:, 'windspeed'] != 0, 'windspeed'].mean()

train_2_2 = data_2_2.loc[train.index, :]
test_2_2 = data_2_2.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [11]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_2_2.drop(columns=['casual', 'registered', 'count']), train_2_2.loc[:, 'registered'])
pred_r = rf_r.predict(test_2_2)

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_2_2.drop(columns=['casual', 'registered', 'count']), train_2_2.loc[:, 'casual'])
pred_c = rf_c.predict(test_2_2)

pred = pred_r + pred_c

In [12]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_2_2.csv', index=False)

### 仮説 2-3 : interpolate
風速 0 を線形補完

In [13]:
data_2_3 = data_0.copy()
data_2_3.replace({'windspeed': {0: np.nan}}, inplace=True)
data_2_3.interpolate(method='time', inplace=True, limit_direction='both')

train_2_3 = data_2_3.loc[train.index, :]
test_2_3 = data_2_3.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [14]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_2_3.drop(columns=['casual', 'registered', 'count']), train_2_3.loc[:, 'registered'])
pred_r = rf_r.predict(test_2_3)

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_2_3.drop(columns=['casual', 'registered', 'count']), train_2_3.loc[:, 'casual'])
pred_c = rf_c.predict(test_2_3)

pred = pred_r + pred_c

In [15]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_2_3.csv', index=False)

### 仮説 2-4 : predict by RF

In [16]:
data_2_4 = data_0.copy()

rf_w = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_w.fit(data_2_4.loc[data_2_4.loc[:, 'windspeed'] != 0, :].drop(columns=['casual', 'registered', 'count', 'windspeed']),
         data_2_4.loc[data_2_4.loc[:, 'windspeed'] != 0, 'windspeed'])
pred_w = rf_w.predict(data_2_4.loc[data_2_4.loc[:, 'windspeed'] == 0, :].drop(columns=['casual', 'registered', 'count', 'windspeed']))

data_2_4.loc[data_2_4.loc[:, 'windspeed'] == 0, 'windspeed'] = pred_w

train_2_4 = data_2_4.loc[train.index, :]
test_2_4 = data_2_4.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [17]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_2_4.drop(columns=['casual', 'registered', 'count']), train_2_4.loc[:, 'registered'])
pred_r = rf_r.predict(test_2_4)

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_2_4.drop(columns=['casual', 'registered', 'count']), train_2_4.loc[:, 'casual'])
pred_c = rf_c.predict(test_2_4)

pred = pred_r + pred_c

In [18]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_2_4.csv', index=False)

### R: 2-1

## 仮説 3 : weather
天候 4 を 3 とする．  
仮説 1, 仮説 2-1 の手法を用いる．

In [22]:
data_3 = data_0.copy()
data_3.drop(columns=['windspeed'], inplace=True)
data_3.replace({'weather': {4: 3}}, inplace=True)

train_3 = data_3.loc[train.index, :]
test_3 = data_3.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [23]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_3.drop(columns=['casual', 'registered', 'count']), train_3.loc[:, 'registered'])
pred_r = rf_r.predict(test_3)

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_3.drop(columns=['casual', 'registered', 'count']), train_3.loc[:, 'casual'])
pred_c = rf_c.predict(test_3)

pred = pred_r + pred_c

In [24]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_3.csv', index=False)

## 仮説 4 : atemp
atemp を使用しない．  
仮説 3 の下で行う．

In [25]:
data_4 = data_0.copy()
data_4.drop(columns=['windspeed', 'atemp'], inplace=True)
data_4.replace({'weather': {4: 3}}, inplace=True)

train_4 = data_4.loc[train.index, :]
test_4 = data_4.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [26]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_4.drop(columns=['casual', 'registered', 'count']), train_4.loc[:, 'registered'])
pred_r = rf_r.predict(test_4)

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_4.drop(columns=['casual', 'registered', 'count']), train_4.loc[:, 'casual'])
pred_c = rf_c.predict(test_4)

pred = pred_r + pred_c

In [27]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_4.csv', index=False)

## 仮説 5 : log
目的変数を対数変換する．log(x + 1)  
仮説 3 の下で行う．

In [28]:
data_5 = data_0.copy()
data_5.drop(columns=['windspeed'], inplace=True)
data_5.replace({'weather': {4: 3}}, inplace=True)

train_5 = data_5.loc[train.index, :]
test_5 = data_5.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [29]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_5.drop(columns=['casual', 'registered', 'count']), np.log1p(train_5.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_5))

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_5.drop(columns=['casual', 'registered', 'count']), np.log1p(train_5.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_5))

pred = pred_r + pred_c

In [30]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_5.csv', index=False)

## 仮説 6 : scaler
連続変数を標準化する．  
仮説 5 の下で行う．

In [31]:
data_6 = data_0.copy()
data_6.drop(columns=['windspeed'], inplace=True)
data_6.replace({'weather': {4: 3}}, inplace=True)

scaler = StandardScaler()
data_6.loc[:, ['temp', 'atemp', 'humidity']] = scaler.fit_transform(data_6.loc[:, ['temp', 'atemp', 'humidity']])

train_6 = data_6.loc[train.index, :]
test_6 = data_6.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [32]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_6.drop(columns=['casual', 'registered', 'count']), np.log1p(train_6.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_6))

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_6.drop(columns=['casual', 'registered', 'count']), np.log1p(train_6.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_6))

pred = pred_r + pred_c

In [33]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_6.csv', index=False)

## 仮説 7 : datetime
datetime について  
仮説 6 の下で行う

### 仮説 7-1 : month

In [34]:
data_7_1 = data_0.copy()
data_7_1.drop(columns=['windspeed', 'month'], inplace=True)
data_7_1.replace({'weather': {4: 3}}, inplace=True)

train_7_1 = data_7_1.loc[train.index, :]
test_7_1 = data_7_1.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [35]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_7_1.drop(columns=['casual', 'registered', 'count']), np.log1p(train_7_1.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_7_1))

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_7_1.drop(columns=['casual', 'registered', 'count']), np.log1p(train_7_1.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_7_1))

In [36]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred_r + pred_c})
submission.to_csv('./data/output/t_7_1.csv', index=False)

### 仮説 7-2 : season

In [3]:
data_7_2 = data_0.copy()
data_7_2.drop(columns=['windspeed', 'season'], inplace=True)
data_7_2.replace({'weather': {4: 3}}, inplace=True)

train_7_2 = data_7_2.loc[train.index, :]
test_7_2 = data_7_2.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [4]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_7_2.drop(columns=['casual', 'registered', 'count']), np.log1p(train_7_2.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_7_2))

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_7_2.drop(columns=['casual', 'registered', 'count']), np.log1p(train_7_2.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_7_2))

pred = pred_r + pred_c

In [5]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_7_2.csv', index=False)

### 仮説 7-3 : weekday

In [6]:
data_7_3 = data_0.copy()
data_7_3.drop(columns=['windspeed', 'weekday'], inplace=True)
data_7_3.replace({'weather': {4: 3}}, inplace=True)

train_7_3 = data_7_3.loc[train.index, :]
test_7_3 = data_7_3.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [7]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_7_3.drop(columns=['casual', 'registered', 'count']), np.log1p(train_7_3.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_7_3))

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_7_3.drop(columns=['casual', 'registered', 'count']), np.log1p(train_7_3.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_7_3))

pred = pred_r + pred_c

In [8]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_7_3.csv', index=False)

### 仮説 7-4 : day

In [10]:
data_7_4 = data_0.copy()
data_7_4.drop(columns=['windspeed'], inplace=True)
data_7_4.replace({'weather': {4: 3}}, inplace=True)
data_7_4.loc[:, 'data'] = data_7_4.index.day

train_7_4 = data_7_4.loc[train.index, :]
test_7_4 = data_7_4.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [11]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_7_4.drop(columns=['casual', 'registered', 'count']), np.log1p(train_7_4.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_7_4))

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_7_4.drop(columns=['casual', 'registered', 'count']), np.log1p(train_7_4.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_7_4))

pred = pred_r + pred_c

In [12]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_7_4.csv', index=False)

### R: 7-1

### t_7_5 : 7-1 + weather 4

In [25]:
data_7_5 = data_0.copy()
data_7_5.drop(columns=['windspeed', 'month'], inplace=True)

train_7_5 = data_7_5.loc[train.index, :]
test_7_5 = data_7_5.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [26]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_7_5.drop(columns=['casual', 'registered', 'count']), np.log1p(train_7_5.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_7_5))

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_7_5.drop(columns=['casual', 'registered', 'count']), np.log1p(train_7_5.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_7_5))

pred = pred_r + pred_c

In [27]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_7_5.csv', index=False)

: 

## 仮説 8 : humidity

### 仮説 8-1 : delete

In [13]:
data_8_1 = data_0.copy()
data_8_1.drop(columns=['windspeed', 'month', 'humidity'], inplace=True)
data_8_1.replace({'weather': {4: 3}}, inplace=True)

train_8_1 = data_8_1.loc[train.index, :]
test_8_1 = data_8_1.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [14]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_8_1.drop(columns=['casual', 'registered', 'count']), np.log1p(train_8_1.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_8_1))

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_8_1.drop(columns=['casual', 'registered', 'count']), np.log1p(train_8_1.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_8_1))

pred = pred_r + pred_c

In [15]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_8_1.csv', index=False)

### 仮説 8-2: mean

In [16]:
data_8_2 = data_0.copy()
data_8_2.drop(columns=['windspeed', 'month'], inplace=True)
data_8_2.replace({'weather': {4: 3}}, inplace=True)
data_8_2.replace({'humidity': {0: data_8_2.loc[data_8_2.loc[:, 'humidity'] != 0, 'humidity'].mean()}}, inplace=True)

train_8_2 = data_8_2.loc[train.index, :]
test_8_2 = data_8_2.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [17]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_8_2.drop(columns=['casual', 'registered', 'count']), np.log1p(train_8_2.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_8_2))

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_8_2.drop(columns=['casual', 'registered', 'count']), np.log1p(train_8_2.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_8_2))

pred = pred_r + pred_c

In [18]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_8_2.csv', index=False)

### 仮説 8-3 : interpolate

In [19]:
data_8_3 = data_0.copy()
data_8_3.drop(columns=['windspeed', 'month'], inplace=True)
data_8_3.replace({'weather': {4: 3}}, inplace=True)

data_8_3.replace({'humidity': 0}, np.nan, inplace=True)
data_8_3.interpolate(method='time', inplace=True, limit_direction='both')

train_8_3 = data_8_3.loc[train.index, :]
test_8_3 = data_8_3.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [20]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_8_3.drop(columns=['casual', 'registered', 'count']), np.log1p(train_8_3.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_8_3))

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_8_3.drop(columns=['casual', 'registered', 'count']), np.log1p(train_8_3.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_8_3))

pred = pred_r + pred_c

In [21]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_8_3.csv', index=False)

### 仮説 8-4 : RF

In [22]:
data_8_4 = data_0.copy()
data_8_4.drop(columns=['windspeed', 'month'], inplace=True)
data_8_4.replace({'weather': {4: 3}}, inplace=True)

rf_h = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_h.fit(data_8_4.loc[data_8_4.loc[:, 'humidity'] != 0, :].drop(columns=['casual', 'registered', 'count', 'humidity']),
            data_8_4.loc[data_8_4.loc[:, 'humidity'] != 0, 'humidity'])
pred_h = rf_h.predict(data_8_4.loc[data_8_4.loc[:, 'humidity'] == 0, :].drop(columns=['casual', 'registered', 'count', 'humidity']))

data_8_4.loc[data_8_4.loc[:, 'humidity'] == 0, 'humidity'] = pred_h

train_8_4 = data_8_4.loc[train.index, :]
test_8_4 = data_8_4.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [23]:
rf_r = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_r.fit(train_8_4.drop(columns=['casual', 'registered', 'count']), np.log1p(train_8_4.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_8_4))

rf_c = RFR(random_state=0, n_jobs=-1, max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400)
rf_c.fit(train_8_4.drop(columns=['casual', 'registered', 'count']), np.log1p(train_8_4.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_8_4))

pred = pred_r + pred_c

In [24]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_8_4.csv', index=False)

### R: 7-1

## 99 : Highperparameter tuning
using Optuna

In [3]:
from sklearn.model_selection import cross_val_score
import optuna

In [4]:
data_99 = data_0.copy()
data_99.drop(columns=['windspeed', 'month'], inplace=True)

train_99 = data_99.loc[train.index, :]
test_99 = data_99.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [5]:
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 1, 1000)
    max_features = trial.suggest_categorical('max_features', ['sqrt','log2', None])
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 1,1000)
    n_estimators =  trial.suggest_int('n_estimators', 1, 1000)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 5)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    regr = RFR(max_depth = max_depth,
               max_features = max_features,
               max_leaf_nodes = max_leaf_nodes,
               n_estimators = n_estimators,
               min_samples_split = min_samples_split,
               min_samples_leaf = min_samples_leaf,
               n_jobs = 2)

    score = cross_val_score(regr,
                            train_99.drop(columns=['casual', 'registered', 'count']),
                            train_99.loc[:, 'count'],
                            cv=5, scoring="r2")
    r2_mean = score.mean()
    print(r2_mean)

    return r2_mean

In [6]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

optimised_rf = RFR(max_depth = study.best_params['max_depth'], 
                   max_features = study.best_params['max_features'],
                   max_leaf_nodes = study.best_params['max_leaf_nodes'],
                   n_estimators = study.best_params['n_estimators'],
                   min_samples_split = study.best_params['min_samples_split'],
                   min_samples_leaf = study.best_params['min_samples_leaf'],
                   n_jobs=2)

if False:
    optimised_rf.fit(train_99.drop(columns=['casual', 'registered', 'count']) ,train_99.loc[:, 'count'])

[I 2023-06-13 21:46:59,551] A new study created in memory with name: no-name-84255b15-cda3-4887-8c47-1a5363fe4eec
[I 2023-06-13 21:47:05,823] Trial 0 finished with value: 0.6205811513204242 and parameters: {'max_depth': 268, 'max_features': 'log2', 'max_leaf_nodes': 216, 'n_estimators': 420, 'min_samples_split': 2, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.6205811513204242.


0.6205811513204242


[I 2023-06-13 21:47:12,124] Trial 1 finished with value: 0.5449165057283483 and parameters: {'max_depth': 8, 'max_features': 'sqrt', 'max_leaf_nodes': 392, 'n_estimators': 554, 'min_samples_split': 2, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.6205811513204242.


0.5449165057283483


[I 2023-06-13 21:47:12,644] Trial 2 finished with value: 0.6534210648188995 and parameters: {'max_depth': 502, 'max_features': None, 'max_leaf_nodes': 162, 'n_estimators': 16, 'min_samples_split': 2, 'min_samples_leaf': 8}. Best is trial 2 with value: 0.6534210648188995.


...

[I 2023-06-13 22:29:05,084] Trial 98 finished with value: 0.7123899641225597 and parameters: {'max_depth': 256, 'max_features': None, 'max_leaf_nodes': 926, 'n_estimators': 914, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 57 with value: 0.7167117775657018.


0.7123899641225597


[I 2023-06-13 22:29:40,124] Trial 99 finished with value: 0.7149334556321394 and parameters: {'max_depth': 164, 'max_features': None, 'max_leaf_nodes': 950, 'n_estimators': 803, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 57 with value: 0.7167117775657018.


0.7149334556321394


RandomForestRegressor(max_depth=167, max_features=None, max_leaf_nodes=969,
                      min_samples_split=3, n_estimators=858, n_jobs=2)

In [7]:
rf_r = RFR(random_state=0, n_jobs=2, max_depth=167, max_features=None, max_leaf_nodes=969, min_samples_split=3, n_estimators=858)
rf_r.fit(train_99.drop(columns=['casual', 'registered', 'count']), np.log1p(train_99.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_99))

rf_c = RFR(random_state=0, n_jobs=2, max_depth=167, max_features=None, max_leaf_nodes=969, min_samples_split=3, n_estimators=858)
rf_c.fit(train_99.drop(columns=['casual', 'registered', 'count']), np.log1p(train_99.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_99))

pred = pred_r + pred_c

In [8]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_99.csv', index=False)

## 仮説 9 : peak time

### 仮説 9-1 : Add for both

In [9]:
data_9_1 = data_0.copy()
data_9_1.drop(columns=['windspeed', 'month'], inplace=True)

data_9_1.loc[:, 'peak'] = 0
peak_df = data_9_1.loc[data_9_1.loc[:, 'workingday'] == 1, :]
peak_df = peak_df.loc[peak_df.loc[:, 'hour'].isin([8, 17, 18]), :]
data_9_1.loc[peak_df.index, 'peak'] = 1

train_9_1 = data_9_1.loc[train.index, :]
test_9_1 = data_9_1.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [10]:
rf_r = RFR(random_state=0, n_jobs=2, max_depth=167, max_features=None, max_leaf_nodes=969, min_samples_split=3, n_estimators=858)
rf_r.fit(train_9_1.drop(columns=['casual', 'registered', 'count']), np.log1p(train_9_1.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_9_1))

rf_c = RFR(random_state=0, n_jobs=2, max_depth=167, max_features=None, max_leaf_nodes=969, min_samples_split=3, n_estimators=858)
rf_c.fit(train_9_1.drop(columns=['casual', 'registered', 'count']), np.log1p(train_9_1.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_9_1))

pred = pred_r + pred_c

In [11]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_9_1.csv', index=False)

### 仮説 9-2 : Add only for registered

In [12]:
rf_r = RFR(random_state=0, n_jobs=2, max_depth=167, max_features=None, max_leaf_nodes=969, min_samples_split=3, n_estimators=858)
rf_r.fit(train_9_1.drop(columns=['casual', 'registered', 'count']), np.log1p(train_9_1.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_9_1))

rf_c = RFR(random_state=0, n_jobs=2, max_depth=167, max_features=None, max_leaf_nodes=969, min_samples_split=3, n_estimators=858)
rf_c.fit(train_9_1.drop(columns=['casual', 'registered', 'count', 'peak']), np.log1p(train_9_1.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_9_1.drop(columns=['peak'])))

pred = pred_r + pred_c

In [13]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_9_2.csv', index=False)

### 仮説 9-3 : trend (8 - 17)

In [14]:
data_9_3 = data_0.copy()
data_9_3.drop(columns=['windspeed', 'month'], inplace=True)

data_9_3.loc[:, 'trend'] = 0
peak_df = data_9_3.loc[data_9_3.loc[:, 'workingday'] == 1, :]
peak_df = peak_df.loc[peak_df.loc[:, 'hour'].isin([x for x in range(8, 18)]), :]
data_9_3.loc[peak_df.index, 'trend'] = 1

train_9_3 = data_9_3.loc[train.index, :]
test_9_3 = data_9_3.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [15]:
rf_r = RFR(random_state=0, n_jobs=2, max_depth=167, max_features=None, max_leaf_nodes=969, min_samples_split=3, n_estimators=858)
rf_r.fit(train_9_3.drop(columns=['casual', 'registered', 'count']), np.log1p(train_9_3.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_9_3))

rf_c = RFR(random_state=0, n_jobs=2, max_depth=167, max_features=None, max_leaf_nodes=969, min_samples_split=3, n_estimators=858)
rf_c.fit(train_9_3.drop(columns=['casual', 'registered', 'count', 'trend']), np.log1p(train_9_3.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_9_3.drop(columns=['trend'])))

pred = pred_r + pred_c

In [16]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_9_3.csv', index=False)

## 仮説 10 : holiday

In [22]:
data_10 = data_0.copy()
data_10.drop(columns=['windspeed', 'month', 'holiday'], inplace=True)

data_10.loc[:, 'trend'] = 0
peak_df = data_10.loc[data_10.loc[:, 'workingday'] == 1, :]
peak_df = peak_df.loc[peak_df.loc[:, 'hour'].isin([x for x in range(8, 18)]), :]
data_10.loc[peak_df.index, 'trend'] = 1

train_10 = data_10.loc[train.index, :]
test_10 = data_10.loc[test.index, :].drop(columns=['casual', 'registered', 'count'])

In [23]:
rf_r = RFR(random_state=0, n_jobs=2, max_depth=167, max_features=None, max_leaf_nodes=969, min_samples_split=3, n_estimators=858)
rf_r.fit(train_10.drop(columns=['casual', 'registered', 'count']), np.log1p(train_10.loc[:, 'registered']))
pred_r = np.expm1(rf_r.predict(test_10))

rf_c = RFR(random_state=0, n_jobs=2, max_depth=167, max_features=None, max_leaf_nodes=969, min_samples_split=3, n_estimators=858)
rf_c.fit(train_10.drop(columns=['casual', 'registered', 'count', 'trend']), np.log1p(train_10.loc[:, 'casual']))
pred_c = np.expm1(rf_c.predict(test_10.drop(columns=['trend'])))

pred = pred_r + pred_c

In [24]:
submission = pd.DataFrame({'datetime': test.index, 'count': pred})
submission.to_csv('./data/output/t_10.csv', index=False)